In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Quantum transfer learning {#quantum_transfer_learning}
=========================

::: {.meta}
:property=\"og:description\": Combine PyTorch and PennyLane to train a
hybrid quantum-classical image classifier using transfer learning.
:property=\"og:image\":
<https://pennylane.ai/qml/_images/transfer_images.png>
:::

*Author: Andrea Mari --- Posted: 19 December 2019. Last updated: 28
January 2021.*

In this tutorial we apply a machine learning method, known as *transfer
learning*, to an image classifier based on a hybrid classical-quantum
network.

This example follows the general structure of the PyTorch [tutorial on
transfer
learning](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)
by Sasank Chilamkurthy, with the crucial difference of using a quantum
circuit to perform the final classification task.

More details on this topic can be found in the research paper \[1\]
([Mari et al. (2019)](https://arxiv.org/abs/1912.08278)).

Introduction
------------

Transfer learning is a well-established technique for training
artificial neural networks (see e.g., Ref. \[2\]), which is based on the
general intuition that if a pre-trained network is good at solving a
given problem, then, with just a bit of additional training, it can be
used to also solve a different but related problem.

As discussed in Ref. \[1\], this idea can be formalized in terms of two
abstract networks $A$ and $B$, independently from their quantum or
classical physical nature.

| 

![](../demonstrations/quantum_transfer_learning/transfer_learning_general.png){.align-center}

| 

As sketched in the above figure, one can give the following **general
definition of the transfer learning method**:

1.  Take a network $A$ that has been pre-trained on a dataset $D_A$ and
    for a given task $T_A$.
2.  Remove some of the final layers. In this way, the resulting
    truncated network $A'$ can be used as a feature extractor.
3.  Connect a new trainable network $B$ at the end of the pre-trained
    network $A'$.
4.  Keep the weights of $A'$ constant, and train the final block $B$
    with a new dataset $D_B$ and/or for a new task of interest $T_B$.

When dealing with hybrid systems, depending on the physical nature
(classical or quantum) of the networks $A$ and $B$, one can have
different implementations of transfer learning as

summarized in following table:

| 

::: {.rst-class}
docstable
:::

  -------------------------------------------------------------------------
  Network A   Network B   Transfer learning scheme
  ----------- ----------- -------------------------------------------------
  Classical   Classical   CC - Standard classical method. See e.g., Ref.
                          \[2\].

  Classical   Quantum     CQ - **Hybrid model presented in this tutorial.**

  Quantum     Classical   QC - Model studied in Ref. \[1\].

  Quantum     Quantum     QQ - Model studied in Ref. \[1\].
  -------------------------------------------------------------------------

Classical-to-quantum transfer learning
--------------------------------------

We focus on the CQ transfer learning scheme discussed in the previous
section and we give a specific example.

1.  As pre-trained network $A$ we use **ResNet18**, a deep residual
    neural network introduced by Microsoft in Ref. \[3\], which is
    pre-trained on the *ImageNet* dataset.
2.  After removing its final layer we obtain $A'$, a pre-processing
    block which maps any input high-resolution image into 512 abstract
    features.
3.  Such features are classified by a 4-qubit \"dressed quantum
    circuit\" $B$, i.e., a variational quantum circuit sandwiched
    between two classical layers.
4.  The hybrid model is trained, keeping $A'$ constant, on the
    *Hymenoptera* dataset (a small subclass of ImageNet) containing
    images of *ants* and *bees*.

A graphical representation of the full data processing pipeline is given
in the figure below.

![](../demonstrations/quantum_transfer_learning/transfer_learning_c2q.png){.align-center}


General setup
=============

::: {.note}
::: {.title}
Note
:::

To use the PyTorch interface in PennyLane, you must first [install
PyTorch](https://pytorch.org/get-started/locally/#start-locally).
:::

In addition to *PennyLane*, we will also need some standard *PyTorch*
libraries and the plotting library *matplotlib*.


In [ ]:
# Some parts of this code are based on the Python script:
# https://github.com/pytorch/tutorials/blob/master/beginner_source/transfer_learning_tutorial.py
# License: BSD





# PyTorch







# Pennylane






# Plotting


# OpenMP: number of parallel threads.

Setting of the main hyper-parameters of the model
=================================================

::: {.note}
::: {.title}
Note
:::

To reproduce the results of Ref. \[1\], `num_epochs` should be set to
`30` which may take a long time. We suggest to first try with
`num_epochs=1` and, if everything runs smoothly, increase it to a larger
value.
:::


We initialize a PennyLane device with a `default.qubit` backend.


We configure PyTorch to use CUDA only if available. Otherwise the CPU is
used.


Dataset loading
===============

::: {.note}
::: {.title}
Note
:::

The dataset containing images of *ants* and *bees* can be downloaded
[here](https://download.pytorch.org/tutorial/hymenoptera_data.zip) and
should be extracted in the subfolder `../_data/hymenoptera_data`.
:::

This is a very small dataset (roughly 250 images), too small for
training from scratch a classical or quantum model, however it is enough
when using *transfer learning* approach.

The PyTorch packages `torchvision` and `torch.utils.data` are used for
loading the dataset and performing standard preliminary image
operations: resize, center, crop, normalize, *etc.*


In [ ]:
# Initialize dataloader





# function to plot images

Let us show a batch of the test data, just to have an idea of the
classification problem.


In [ ]:
# Get a batch of training data


# Make a grid from batch

Variational quantum circuit
===========================

We first define some quantum layers that will compose the quantum
circuit.


Now we define the quantum circuit through the PennyLane
[qnode]{.title-ref} decorator .

The structure is that of a typical variational quantum circuit:

-   **Embedding layer:** All qubits are first initialized in a balanced
    superposition of *up* and *down* states, then they are rotated
    according to the input parameters (local embedding).
-   **Variational layers:** A sequence of trainable rotation layers and
    constant entangling layers is applied.
-   **Measurement layer:** For each qubit, the local expectation value
    of the $Z$ operator is measured. This produces a classical output
    vector, suitable for additional post-processing.


Dressed quantum circuit
=======================

We can now define a custom `torch.nn.Module` representing a *dressed*
quantum circuit.

This is a concatenation of:

-   A classical pre-processing layer (`nn.Linear`).
-   A classical activation function (`torch.tanh`).
-   A constant `np.pi/2.0` scaling.
-   The previously defined quantum circuit (`quantum_net`).
-   A classical post-processing layer (`nn.Linear`).

The input of the module is a batch of vectors with 512 real parameters
(features) and the output is a batch of vectors with two real outputs
(associated with the two classes of images: *ants* and *bees*).


Hybrid classical-quantum model
==============================

We are finally ready to build our full hybrid classical-quantum network.
We follow the *transfer learning* approach:

1.  First load the classical pre-trained network *ResNet18* from the
    `torchvision.models` zoo.
2.  Freeze all the weights since they should not be trained.
3.  Replace the last fully connected layer with our trainable dressed
    quantum circuit (`DressedQuantumNet`).

::: {.note}
::: {.title}
Note
:::

The *ResNet18* model is automatically downloaded by PyTorch and it may
take several minutes (only the first time).
:::


In [ ]:
# Notice that model_hybrid.fc is the last layer of ResNet18


# Use CUDA or CPU according to the "device" object.

Training and results
====================

Before training the network we need to specify the *loss* function.

We use, as usual in classification problem, the *cross-entropy* which is
directly available within `torch.nn`.


We also initialize the *Adam optimizer* which is called at each training
step in order to update the weights of the model.


We schedule to reduce the learning rate by a factor of
`gamma_lr_scheduler` every 10 epochs.


What follows is a training function that will be called later. This
function should return a trained model that can be used to make
predictions (classifications).


We are ready to perform the actual training process.


Visualizing the model predictions
=================================


We first define a visualization function for a batch of test data.


Finally, we can run the previous function to see a batch of images with
the corresponding predictions.


References
==========

\[1\] Andrea Mari, Thomas R. Bromley, Josh Izaac, Maria Schuld, and
Nathan Killoran. *Transfer learning in hybrid classical-quantum neural
networks*. arXiv:1912.08278 (2019).

\[2\] Rajat Raina, Alexis Battle, Honglak Lee, Benjamin Packer, and
Andrew Y Ng. *Self-taught learning: transfer learning from unlabeled
data*. Proceedings of the 24th International Conference on Machine
Learning\*, 759--766 (2007).

\[3\] Kaiming He, Xiangyu Zhang, Shaoqing ren and Jian Sun. *Deep
residual learning for image recognition*. Proceedings of the IEEE
Conference on Computer Vision and Pattern Recognition, 770-778 (2016).

\[4\] Ville Bergholm, Josh Izaac, Maria Schuld, Christian Gogolin,
Carsten Blank, Keri McKiernan, and Nathan Killoran. *PennyLane:
Automatic differentiation of hybrid quantum-classical computations*.
arXiv:1811.04968 (2018).

About the author
================
